In [1]:
# 1) import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from pprint import pprint
import datetime as DT
import requests
import json
import re


In [2]:
# 2) Create a path to chrome driver: Make sure to change the path!
executable_path = {'executable_path': '../chromedriver'}
# 3) load the path into browser 4) initialize the browser
browser = Browser('chrome', **executable_path, headless=True)


# E of ETL Part 1: Data Extraction via API Call for Bestbuy

In [3]:
url = 'https://api.bestbuy.com/v1/products((categoryPath.id=abcat0502000))?apiKey=BC9ll4HTWIkIUFAUJjZFoKq3&show=manufacturer,modelNumber,name,regularPrice,salePrice,details.name,details.value,shortDescription,upc,url&pageSize=100&page=1&format=json'

# for product list:
response = requests.get(url)
data = response.json()
data

{'from': 1,
 'to': 100,
 'currentPage': 1,
 'total': 841,
 'totalPages': 9,
 'queryTime': '0.024',
 'totalTime': '0.354',
 'partial': False,
 'canonicalUrl': '/v1/products((categoryPath.id=abcat0502000))?show=manufacturer,modelNumber,name,regularPrice,salePrice,details.name,details.value,shortDescription,upc,url&pageSize=100&format=json&apiKey=BC9ll4HTWIkIUFAUJjZFoKq3',
 'products': [{'manufacturer': 'Acer',
   'modelNumber': 'C740-C4PE',
   'name': 'Acer - 11.6" Chromebook - Intel Celeron - 4GB Memory - 16GB Solid State Drive - Gray',
   'regularPrice': 249.99,
   'salePrice': 249.99,
   'details': [{'name': 'Cache Memory',
     'value': '2 megabytes',
     'values': ['2 megabytes']},
    {'name': 'Screen Size', 'value': '11.6 inches', 'values': ['11.6 inches']},
    {'name': 'Hard Drive Capacity',
     'value': '16 gigabytes',
     'values': ['16 gigabytes']},
    {'name': 'Wireless Networking',
     'value': 'Wireless-A',
     'values': ['Wireless-A',
      'Wireless-AC',
      'Wir

# T of ETL Part 1: Data Extraction via API Call for Bestbuy

In [4]:
list_one = []
for x in range(1, 10):
    url = f'https://api.bestbuy.com/v1/products((categoryPath.id=abcat0502000))?apiKey=BC9ll4HTWIkIUFAUJjZFoKq3&show=manufacturer,modelNumber,upc,name,regularPrice,salePrice,url,details.name,details.value,shortDescription&pageSize=100&page={x}&format=json'
    response = requests.get(url)
    data = response.json()
    list_one.extend(data.get("products"))
    print(x)

    

1
2
3
4
5
6
7
8
9


In [5]:
productlist = list_one

bestbuy_laptops = []
  
    
for product in productlist:
    title = product['name']
    
    regularprice = product['regularPrice']
    saleprice = product['salePrice']
    if saleprice:  # is not null:
        price = saleprice
    else: 
        price = regularprice
    
    link = product['url']
    upc = product['upc']
    model = product['modelNumber']
    brand = product['manufacturer']
       
    for info in product['details']:
                
        # RAM in GB:
        if info['name'] == 'System Memory (RAM)':
            ram = re.findall('\d+', info['value'])[0]

        # in inches:
        if info['name'] == 'Screen Size':
            raw_ss = info['value']
            raw_ss = raw_ss[0: raw_ss.find(' ')]
            screensize = raw_ss.strip()

        # HD in GB:
        if info['name'] == 'Hard Drive Capacity':
            hd = re.findall('\d+', info['value'])[0]
        
        # CPU:
        if info['name'] == 'Processor Model':
            cpu = info['value']
            
            
    dic = {
        'brand' : brand,
        'model': model,
        'screensize': screensize,
        'cpu': cpu,
        'ram': ram,
        'hd': hd,
        'upc': upc,
        'price': price,
        'title': title,
        'link': link
    }
    
    bestbuy_laptops.append(dic)



In [6]:
bestbuy_df = {}
bestbuy_df = pd.DataFrame(bestbuy_laptops)
bestbuy_df = bestbuy_df[['brand', 'model', 'screensize', 'cpu', 'ram', 'hd', 'upc', 'price', 'link', 'title']]

# Add the load date:
now = DT.datetime.now()

bestbuy_df['lastUpdated'] = now.strftime("%Y-%m-%d %H:%M")

final_bestbuy_df = bestbuy_df.dropna(subset=['upc'])
final_bestbuy_df.head()



,brand,model,screensize,cpu,ram,hd,upc,price,link,title,lastUpdated
0,Acer,C740-C4PE,11.6,Intel Celeron,4,16,887899773891,249.99,https://api.bestbuy.com/click/-/4020462/pdp,"Acer - 11.6"" Chromebook - Intel Celeron - 4GB ...",2019-10-02 12:12
1,Apple,MNYF2LL/A,12,Intel Core m3,8,16,190198202536,1299.99,https://api.bestbuy.com/click/-/4598800/pdp,"Apple - MacBook® - 12"" Display - Intel Core M3...",2019-10-02 12:12
2,Apple,MNYJ2LL/A,12,Intel 7th Generation Core i5,8,16,190198203793,1599.99,https://api.bestbuy.com/click/-/4599400/pdp,"Apple - MacBook® - 12"" Display - Intel Core i5...",2019-10-02 12:12
3,Apple,MV992LL/A,13.3,Intel 8th Generation Core i5,8,16,190199114456,1799.99,https://api.bestbuy.com/click/-/4599600/pdp,"Apple - MacBook Pro - 13"" Display with Touch B...",2019-10-02 12:12
4,Apple,MV962LL/A,13.3,Intel 8th Generation Core i5,8,16,190199112216,1799.99,https://api.bestbuy.com/click/-/4599700/pdp,"Apple - MacBook Pro - 13"" Display with Touch B...",2019-10-02 12:12


# E of ETL Part 2: Data Extraction via Webscraping for FRY'S

In [7]:
###############################################################
# EXTRACTION WITH TRANSFORMATION:
###############################################################

#FRYS:
###############################################################
# 5) initialize the page parameters and the first site visit:
###############################################################
next_page = "https://www.frys.com/search?cat=-73060&nearbyStoreName=false&pType=pDisplay&rows=20&resultpage=0&start=0&rows=20"
page_remaining = True
page_no = 0
increment = 20
start_no = 0
# 6) make a variable that holds the url we want to scrape data from
laptops = {}

frys_laptops = []

# 7) using the browser function, visit the webpage
browser.visit(next_page)
# 8) stop the scrape process for 1.25 seconds using the time function. This allows the webpage to load in full
time.sleep(1.25)

# 9) create the html variable that allows the browser to visit the HTML elements on the webpage
html = browser.html
# 10) initialize the BeautifulSoup module by telling it to parse html from the HTML elements on the webpage.
soup = bs(html, 'html.parser')


# 11) loop through until no products are found:
###############################################################
while page_remaining == True:
    
    # 12) Data retrieval logic: 
    ###############################################################
    try:
        mothers = soup.find_all('div', class_='togrid')
        for mother in mothers:
            title = mother.find('p', class_='productDescp').get_text().strip()
            
            raw_price = mother.find('label', class_='red_txt').get_text().strip()
            text_price = raw_price.replace("$", "")
            nocommas = text_price.replace(",", "")
            price = float(nocommas)
            
            partial_link = mother.find('p', class_='productDescp').find('a')['href'].strip()
            base_url = "https://www.frys.com/"
            link = base_url + partial_link
            
            results = mother.find_all('div', class_='prodModel')
            for result in results:
                new_list = result.find_all('p', class_='mar-btm')
                for p in new_list:
                    dataitem = p.text.replace(" ", "")
                    di = dataitem.replace('\n', "")
                    dataitem_split = di.split(":")
                    if dataitem_split[0] == "Brand":
                        brand = dataitem_split[1]
                    if dataitem_split[0] == "UPC":
                        upc = dataitem_split[1]
                    if dataitem_split[0] == "Model":
                        model = dataitem_split[1]
                        
            # 13) create a new dictionary that will hold the data that we scrape
            dic = {
                'title': title,
                'price': price,
                'link': link,
                'upc': upc,
                'model': model,
                'brand': brand
            }
            print(dic)
            frys_laptops.append(dic)
            
    except AttributeError:
        # errors out when there are 2 prices in red (price with rebate)
        print("error found. skipping...")
    
    
    # 14) then process the Next Page logic:
    ###############################################################
    page_no += 1
    start_no = start_no + increment
    next_page = f'https://www.frys.com/search?cat=-73060&nearbyStoreName=false&pType=pDisplay&rows=20&resultpage={page_no}&start={start_no}&rows=20'

    browser.visit(next_page)
    time.sleep(3)

    html = browser.html
    soup = bs(html, 'html.parser') 
    
    next_page_avail = soup.find_all('p')
    for p in next_page_avail:
        message = p.text
        if message == " No products were found that matched your search":
            page_remaining = False
        


{'title': 'HP PB 6470B 14" Refurbished Laptop With Intel Ci5-3210M Processor, 8GB Memory, 500GB Hard Drive, DVD-ROM, Windows 10', 'price': 399.0, 'link': 'https://www.frys.com/product/9800005;jsessionid=D560CC4491963F1CE0A8D22CF83F6206.node3?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897751172', 'model': 'PB6470BCi5-3210M', 'brand': 'HP'}
{'title': 'Lenovo TP 11e 11.6" Refurbished Chromebook With Intel Celeorn N2920 Processor, 4GB Memory, 16GB Storage, Chrome OS', 'price': 199.0, 'link': 'https://www.frys.com/product/9914936;jsessionid=D560CC4491963F1CE0A8D22CF83F6206.node3?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897764240', 'model': 'TP11eCel.N2920', 'brand': 'Lenovo'}
{'title': 'HP EB 8440P 14" Refurbished Laptop With Intel Ci5-520M Processor, 4GB Memory, 160GB Hard Drive, DVD Rom, Windows 10', 'price': 299.0, 'link': 'https://www.frys.com/product/9911496;jsessionid=D560CC4491963F1CE0A8D22CF83F6206.node3?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897763816', 'model': 'EB8440Pi5-520M', '

{'title': 'HP 15-da0021cy 15.6" Refurbished Touchscreen Laptop With Intel Ci5-8250U Processor, 8GB Memory, 16GB Optane Memory, 1TB Hard Drive, Windows 10 - Rose Gold', 'price': 499.0, 'link': 'https://www.frys.com/product/9914926?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '193015924024', 'model': '15-da0021i5-8250U', 'brand': 'HP'}
{'title': 'Dell i5584-7851SLV-PUS 15.6" Laptop With Intel Ci7-8565U Processor, 8GB Memory, 256GB SSD , Windows 10', 'price': 849.0, 'link': 'https://www.frys.com/product/9876535?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116314851', 'model': 'Insp155000i78565', 'brand': 'Dell'}
{'title': 'Dell i5481-3236GRY-PUS 14" 2-in-1 Touchscreen Laptop With Intel i3-8145U Processor, 4GB Memory, 128GB SSD, Windows 10S', 'price': 419.0, 'link': 'https://www.frys.com/product/9851315?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116313236', 'model': 'Insp145481i3-8145U', 'brand': 'Dell'}
{'title': 'Samsung NP930SBE-K01US 13.3" 2-in-1 Touchscreen Laptop With Intel Ci7-8565U Processor, 8G

{'title': 'HP EB 9480m 14" Refurbished Laptop With Intel Ci5-4300U Processor, 8GB Memory, 512GB SSD, Windows 10', 'price': 368.0, 'link': 'https://www.frys.com/product/9841105?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897751295', 'model': 'EB9480mi5-4300U', 'brand': 'HP'}
{'title': 'LG 15Z970S 15.6" Laptop With Ci5-7200U Processor, 8GB Memory, 256GB SSD, Windows 10', 'price': 499.0, 'link': 'https://www.frys.com/product/9078257?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '719192611115', 'model': '15Z970-U.AAS5U1', 'brand': 'LG'}
{'title': 'Samsung Odyesse Z 15.6" Gaming Laptop with Intel Core i7-8750H, 16GB, 256GBSSD,GTX1060 and Windows 10 NP850XAC-X01US', 'price': 1199.0, 'link': 'https://www.frys.com/product/9661423?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '887276266169', 'model': 'OdysseyZ', 'brand': 'Samsung'}
{'title': 'Acer A515-51 15.6" Laptop With Intel Ci7-8550U Processor, 12GB Memory, 1TB Hard Drive+128GB SSD, Windows 10', 'price': 899.0, 'link': 'https://www.frys.com/product/9657543?sit

{'title': 'Asus F441BA 14" Laptop With AMD A9-9425 Processor, 8GB Memory, 256GB SSD and Windows 10', 'price': 371.07, 'link': 'https://www.frys.com/product/9721334?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '192876040553', 'model': 'F441BA-DS95', 'brand': 'Asus'}
{'title': 'Asus GL503GE-RS71 15.6" Gaming Laptop With Intel Ci7-8750H Processor, 8GB Memory, 1TB Hard Drive, GTX 1050TI 4GB Graphics, Windows 10', 'price': 799.0, 'link': 'https://www.frys.com/product/9528002?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '889349993576', 'model': 'GL503GE-RS71', 'brand': 'Asus'}
{'title': 'Asus F505ZA 15.6" Laptop With AMD R3-22200U Processor, 6GB Memory, 1TB Hard Drive and Windows 10', 'price': 364.07, 'link': 'https://www.frys.com/product/9711374?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '192876018637', 'model': 'F505ZA-DB31', 'brand': 'Asus'}
{'title': 'Dell i5567-7291GRY, 15.6" Touchscreen Laptop With Intel Core i7 Processor, 16GB Memory,AMD Radeon R7 M445 4G DDR5 Graphics Card, 1TB Hard Drive and Windows 10',

{'title': 'HP 17-y028 AMD A12-9700P 17.3" Touchscreen Laptop With 12GB Memory, 2TB Hard Drive, Windows 10 - Green(Refurbished)', 'price': 414.07, 'link': 'https://www.frys.com/product/9205599?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '190781661085', 'model': '17-y028', 'brand': 'HP'}
{'title': 'Dell Inspiron 11 Intel Core m3-6Y30 11.6" 2 in 1 Touchscreen Red Laptop With 4GB Memory, 500GB Hard Drive, Windows 10', 'price': 324.57, 'link': 'https://www.frys.com/product/9120208?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116217282', 'model': 'i3169-0011RED', 'brand': 'Dell'}
{'title': 'Lenovo Ideapad 110  15.6" Laptop With Intel Pentium N3710 4GB Memory, 1TB Hard Drive, Windows 10', 'price': 156.57, 'link': 'https://www.frys.com/product/9019117?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '190793580527', 'model': '80T700ACUS', 'brand': 'Lenovo'}
{'title': 'HP 17-x173, 17.3" Refurbished Laptop With Intel® Core™ i7 Processor, 8GB Memory, 1TB Hard Drive and Windows 10', 'price': 499.0, 'link': 'https://www.f

{'title': 'Dell Insp.17 Ci7-8550U 17.3" Screen Size, 8GB,1TB+16GB Optane DVDRW, Windows 10', 'price': 633.97, 'link': 'https://www.frys.com/product/9560252?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116294375', 'model': 'i5770-7375SLV-PUS', 'brand': 'Dell'}
{'title': 'Dell Insprion 11 Intel Pentium N3710, 11.6" Touchscreen Convertible Laptop With 4GB Memory, 500GB Hard Drive, Windows 10', 'price': 277.0, 'link': 'https://www.frys.com/product/8860292?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116217367', 'model': 'i3168-3271BLU', 'brand': 'Dell'}
{'title': 'Asus S510UA-DS71 15.6" Laptop With Intel Ci7-8550U Processor, 8GB Memory,1TB Hard Drive + 128GB SSD, Full HD, Windows 10', 'price': 599.0, 'link': 'https://www.frys.com/product/9466921?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '889349896570', 'model': 'S510UA-DS71', 'brand': 'Asus'}
{'title': 'Lenovo Ideapad 100 Intel Core i5-5200U, 15.6" Laptop With 4GB Memory, 1TB Hard Drive, Windows 10', 'price': 299.0, 'link': 'https://www.frys.com/produc

# Data Transformation: Fry's

In [8]:
# create a dataframe from the list of dictionaries:
frys_df = pd.DataFrame(frys_laptops)
frys_df
# print(len(frys_df))


,brand,link,model,price,title,upc
0,HP,https://www.frys.com/product/9800005;jsessioni...,PB6470BCi5-3210M,399.00,"HP PB 6470B 14"" Refurbished Laptop With Intel ...",696897751172
1,Lenovo,https://www.frys.com/product/9914936;jsessioni...,TP11eCel.N2920,199.00,"Lenovo TP 11e 11.6"" Refurbished Chromebook Wit...",696897764240
2,HP,https://www.frys.com/product/9911496;jsessioni...,EB8440Pi5-520M,299.00,"HP EB 8440P 14"" Refurbished Laptop With Intel ...",696897763816
3,Lenovo,https://www.frys.com/product/9914966;jsessioni...,TPX240i5-4300U,399.00,"Lenovo TP X240 12.5"" Refurbished Laptop With I...",696897764264
4,Lenovo,https://www.frys.com/product/9721554;jsessioni...,T420Ci5-2520M,349.00,"Lenovo T420 14.1"" Refurbished Laptop With Ci5-...",696897743108
5,Lenovo,https://www.frys.com/product/9914956;jsessioni...,N21CeleronN2840,167.00,"Lenovo N21 11.6"" Refurbished Chromebook With I...",696897764257
6,Lenovo,https://www.frys.com/product/9797874;jsessioni...,X131EIntel1007U,249.00,"Lenovo X131E 11.6"" Refurbished Laptop With Int...",696897751226
7,Lenovo,https://www.frys.com/product/9779764;jsessioni...,TPT430i5-3210M,449.00,"Lenovo Thinkpad T430 14"" Refurbished Laptop Wi...",696897750199
8,Lenovo,https://www.frys.com/product/9721544;jsessioni...,FY696897743115,299.00,"Lenovo T410 14.1"" Refurbished Laptop With Inte...",696897743115
9,HP,https://www.frys.com/product/9941276;jsessioni...,ChromeBkG4N2840,149.00,"HP G4 11.6"" Refurbished Chromebook With Intel ...",724962748779


In [9]:
frys_df['modified_title'] = frys_df['title'].str.replace('"', '",')
frys_df['modified_title'].head()

0    HP PB 6470B 14", Refurbished Laptop With Intel...
1    Lenovo TP 11e 11.6", Refurbished Chromebook Wi...
2    HP EB 8440P 14", Refurbished Laptop With Intel...
3    Lenovo TP X240 12.5", Refurbished Laptop With ...
4    Lenovo T420 14.1", Refurbished Laptop With Ci5...
Name: modified_title, dtype: object

In [10]:
# for easier parsing:

frownum1 = 0

for x in frys_df['modified_title']:
    new_title = x[x.find(' '):]
    new_title = new_title[new_title.find(' '):]
    new_title = new_title[new_title.find(' '):]
    new_title = new_title.replace('Refurbished ', '')
    new_title = new_title.replace('', '')
    new_title = new_title.replace('-inch', '",')
    new_title = new_title.replace(' Measured Diagonal', '",')
    
    frys_df.loc[frownum1, 'modified_title'] = new_title
        
    frownum1 +=1

frys_df['modified_title']


0       PB 6470B 14", Laptop With Intel Ci5-3210M Pro...
1       TP 11e 11.6", Chromebook With Intel Celeorn N...
2       EB 8440P 14", Laptop With Intel Ci5-520M Proc...
3       TP X240 12.5", Laptop With Intel Ci5-4300u Pr...
4       T420 14.1", Laptop With Ci5-2520M Processor, ...
5       N21 11.6", Chromebook With Intel Celeron N284...
6       X131E 11.6", Laptop With Intel Celeron 1007U ...
7       Thinkpad T430 14", Laptop With Intel Ci5-3210...
8       T410 14.1", Laptop With Intel Ci5-520M Proces...
9       G4 11.6", Chromebook With Intel Celeron N2840...
10      E6440 14", Laptop With Intel Ci5-4200M Proces...
11      9470 14", Laptop With Intel Ci5-3427U Process...
12      i5575-A217SLV-PUS 15.6", FHD Laptop With AMD ...
13      E7440 14", Touchscreen Laptop With Intel Ci5-...
14      TP T440P 14", Laptop With Intel Ci5-4300M Pro...
15      E5440 14", Laptop With Intel Ci5-4300U Proces...
16      TP X230 12.5", Laptop With Intel Ci5-3320M Pr...
17      i5485-A094SLV-PUS 14", 

In [11]:
# UPC FIRST because the rest is such a mess:
frys_df['raw_specs'] = frys_df['upc'] + ', ' + frys_df['modified_title']
frys_df.head()

,brand,link,model,price,title,upc,modified_title,raw_specs
0,HP,https://www.frys.com/product/9800005;jsessioni...,PB6470BCi5-3210M,399.0,"HP PB 6470B 14"" Refurbished Laptop With Intel ...",696897751172,"PB 6470B 14"", Laptop With Intel Ci5-3210M Pro...","696897751172, PB 6470B 14"", Laptop With Intel..."
1,Lenovo,https://www.frys.com/product/9914936;jsessioni...,TP11eCel.N2920,199.0,"Lenovo TP 11e 11.6"" Refurbished Chromebook Wit...",696897764240,"TP 11e 11.6"", Chromebook With Intel Celeorn N...","696897764240, TP 11e 11.6"", Chromebook With I..."
2,HP,https://www.frys.com/product/9911496;jsessioni...,EB8440Pi5-520M,299.0,"HP EB 8440P 14"" Refurbished Laptop With Intel ...",696897763816,"EB 8440P 14"", Laptop With Intel Ci5-520M Proc...","696897763816, EB 8440P 14"", Laptop With Intel..."
3,Lenovo,https://www.frys.com/product/9914966;jsessioni...,TPX240i5-4300U,399.0,"Lenovo TP X240 12.5"" Refurbished Laptop With I...",696897764264,"TP X240 12.5"", Laptop With Intel Ci5-4300u Pr...","696897764264, TP X240 12.5"", Laptop With Inte..."
4,Lenovo,https://www.frys.com/product/9721554;jsessioni...,T420Ci5-2520M,349.0,"Lenovo T420 14.1"" Refurbished Laptop With Ci5-...",696897743108,"T420 14.1"", Laptop With Ci5-2520M Processor, ...","696897743108, T420 14.1"", Laptop With Ci5-252..."


In [12]:
frys_specs = []
frys_specs = frys_df['raw_specs'].str.split(', ', expand = True)
frys_specs.head()
frys_specs[160:]

,0,1,2,3,4,5,6,7,8
160,190940821633,"A12 12.2""",2-in-1 Laptop with Halo Keyboard Intel® Atom™ ...,2GB RAM,32GB ROM - Rose Gold,None,None,None,None
161,889349201862,"ROG 17.3""",Intel i7 Full HD Gaming Laptop,None,None,None,None,None,None
162,889349313107,X555DA 15.6 Inch Laptop AMD A10,8 GB,1TB HDD,Windows 10,None,None,None,None
163,190780345191,14-an090nr,"14""",Laptop With AMD Quad Core Processor,4GB Memory,500GB Hard Drive and Windows 10,None,None,None
164,884116253822,"ALW15-12 Ci5-6300HQ 15.6"",""",8GB,1TB,GTX 1060 6GB Windows 10,None,None,None,None
165,889349406816,"13.3""",Zenbook Flip Intel M3 8GB 512GB SSD Touchscree...,None,None,None,None,None,None
166,889349338568,VivoBook Intel Pentium Quad Core 4GB,"128GB eMMC 14""",Full HD Laptop - Metallic Gray,None,None,None,None,None
167,190780864111,m6-aq105 Intel Core i7-7500U,"15.6""",Touchscreen Laptop With 16GB Memory,1TB Hard Drive,Windows 10,None,None,None
168,191114639825,A315-41-R8UU,"15.6""",Laptop With AMD Ryzen 5 Processor,8GB Memory,1TB Hard Drive and Windows 10 Home,None,None,None
169,190781118817,11-v002,"11.6""",Chromebook With Intel Celeron N3060 Processor,4GB Memory,16GB Solid State Drive and Chrome OS,None,None,None


## Screen Size

In [13]:
# first found in col 1:
fraw_size = []
f_size = []

fraw_size1 = frys_specs[1]

for x in fraw_size1:
    if (x.find('"') > 0):
        # find the first instance of "
        fsize1 = x[0:x.find('"')]
        # find the last space before the "
        fsize2 = fsize1[fsize1.rfind(' ')+1:]
        try:
            firstchar = int(fsize2[0])
            f_size.append(float(fsize2))
        except:
            f_size.append('')
    else:
        f_size.append('')

# len(f_size)
# f_size

In [14]:
# round 2: found in col 2:
f_size2 = []
frownum2 = 0

for x in f_size:
    try:
        if (x == '' or x is None):
            new_size = frys_specs.loc[frownum2, 2]
            new_size2 = new_size.lstrip()
            if (new_size2.find('"') > 0):
                try:
                    firstchar = int(new_size[0])
                    new_size = new_size[0:new_size.find('"')]
                    new_size = new_size[new_size.rfind(' ')+1:]
                    f_size2.append(float(new_size))
                except:
                    f_size2.append(x)
            else:
                f_size2.append(x)
        else:
            f_size2.append(x)
    
    except:
        f_size2.append('')
        
    frownum2 += 1
       
f_size2
    



[14.0,
 11.6,
 14.0,
 12.5,
 14.1,
 11.6,
 11.6,
 14.0,
 14.1,
 11.6,
 14.0,
 14.0,
 15.6,
 14.0,
 14.0,
 14.0,
 12.5,
 14.0,
 17.3,
 15.6,
 15.6,
 15.6,
 15.6,
 11.6,
 15.6,
 15.6,
 15.6,
 14.0,
 17.3,
 15.6,
 15.6,
 12.5,
 14.0,
 17.3,
 14.0,
 12.5,
 15.6,
 17.3,
 14.0,
 15.6,
 15.6,
 15.6,
 14.0,
 13.3,
 13.3,
 15.6,
 15.6,
 15.6,
 14.0,
 14.0,
 13.3,
 15.6,
 15.6,
 11.6,
 12.2,
 11.6,
 15.6,
 14.0,
 11.6,
 15.6,
 17.3,
 11.6,
 14.0,
 15.6,
 15.6,
 11.6,
 15.6,
 17.3,
 15.6,
 15.6,
 15.6,
 15.6,
 14.0,
 11.6,
 13.3,
 14.0,
 15.6,
 15.6,
 13.5,
 14.0,
 14.0,
 15.6,
 15.6,
 15.6,
 13.3,
 15.6,
 12.5,
 14.1,
 15.6,
 14.0,
 15.6,
 15.6,
 15.6,
 15.6,
 15.6,
 13.5,
 13.5,
 15.6,
 17.3,
 13.3,
 11.6,
 15.6,
 13.9,
 11.6,
 11.6,
 15.6,
 14.0,
 11.6,
 12.5,
 14.0,
 15.6,
 15.6,
 14.0,
 15.6,
 15.6,
 15.6,
 15.6,
 14.0,
 15.6,
 15.6,
 15.6,
 12.5,
 '',
 13.3,
 15.6,
 15.6,
 12.5,
 11.6,
 15.6,
 15.6,
 12.3,
 15.6,
 14.0,
 15.6,
 15.6,
 15.6,
 17.3,
 17.3,
 17.3,
 15.6,
 '',
 13.5,
 14.0,
 14

In [15]:
# round 3: lastly found in col 3:
frys_size = []
frownum2 = 0

for x in f_size2:
    try:
        if (x == '' or x is None):
            new_size = frys_specs.loc[frownum2, 3]
            new_size2 = new_size.lstrip()
            if (new_size2.find('"') > 0):
                try:
                    firstchar = int(new_size[0])
                    new_size = new_size[0:new_size.find('"')]
                    new_size = new_size[new_size.rfind(' ')+1:]
                    frys_size.append(float(new_size))
                except:
                    frys_size.append(x)
            else:
                frys_size.append(x)
        else:
            frys_size.append(x)
    
    except:
        frys_size.append('')
    
    frownum2 += 1
        
    

frys_size
    

frys_specs['screensize'] = frys_size

frys_specs


,0,1,2,3,4,5,6,7,8,screensize
0,696897751172,"PB 6470B 14""",Laptop With Intel Ci5-3210M Processor,8GB Memory,500GB Hard Drive,DVD-ROM,Windows 10,None,None,14
1,696897764240,"TP 11e 11.6""",Chromebook With Intel Celeorn N2920 Processor,4GB Memory,16GB Storage,Chrome OS,None,None,None,11.6
2,696897763816,"EB 8440P 14""",Laptop With Intel Ci5-520M Processor,4GB Memory,160GB Hard Drive,DVD Rom,Windows 10,None,None,14
3,696897764264,"TP X240 12.5""",Laptop With Intel Ci5-4300u Processor,8GB Memory,500GB Hard Drive,Windows 10,None,None,None,12.5
4,696897743108,"T420 14.1""",Laptop With Ci5-2520M Processor,4GB Memory,250GB Hard Drive and Windows 10,None,None,None,None,14.1
5,696897764257,"N21 11.6""",Chromebook With Intel Celeron N2840 Processor,4GB Memory,16GB Storage,Chrome OS,None,None,None,11.6
6,696897751226,"X131E 11.6""",Laptop With Intel Celeron 1007U Processor,4GB Memory,320GB Hard Drive,Windows 10 Home,None,None,None,11.6
7,696897750199,"Thinkpad T430 14""",Laptop With Intel Ci5-3210M Processor,8GB Memory,240GB SSD,Windows 10 Pro,None,None,None,14
8,696897743115,"T410 14.1""",Laptop With Intel Ci5-520M Processor,4GB Memory,160GB Hard Drive and Windows 10,None,None,None,None,14.1
9,724962748779,"G4 11.6""",Chromebook With Intel Celeron N2840 Processor,2GB Memory,16GB SSD,Chrome OS,None,None,None,11.6


## CPU

In [16]:
# cpu round 1: col 1:
fraw_cpu = []
fcpu = []

fraw_cpu = frys_specs[1]

for x in fraw_cpu:
    # remove 'processor':
    f_cpu = x.replace(' Processor', '')
    
    # get rid of the screensize:
    if f_cpu.find('"') > 0:
        f_cpu = f_cpu[0:f_cpu.rfind(' ')]
       
    
    
    if f_cpu.find('Intel') > 0:
        f_cpu = f_cpu[f_cpu.find('Intel'):]
        fcpu.append(f_cpu)

    elif f_cpu.find('AMD') > 0:
        f_cpu = f_cpu[f_cpu.find('AMD'):]
        fcpu.append(f_cpu)
    
    else:
        fcpu.append('')

fcpu
        


['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Intel Celeron',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Intel Celeron',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Intel QC',
 '',
 'Intel Core i7-6500U',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Intel® Core™ Core i7-7700HQ NVIDIA GeForce® GTX1070 16GB Memory Gaming Laptop',
 '',
 '',
 'AMD A12-9700P',
 'Intel Core m3-6Y30',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'AMD A10',
 '',
 '',
 '',
 '

In [17]:
# cpu round 2:
fcpu1 = []
frnum = 0

# for x from the first round:
for x in fcpu:
    if x == '':
        f_cpu1 = frys_specs.loc[frnum, 2]
        
        # if it says 'Processor' in it:
        if f_cpu1.find('Processor') > 0:
            # cpu info ends at the first instance of 'Processor':
            f_cpu1 = f_cpu1[0:f_cpu1.find(' Processor')]
            
            # cpu info should start after 'With'
            f_cpu1 = f_cpu1.replace('With', 'with')
            f_cpu1 = f_cpu1[f_cpu1.find('with'):]

            # clean the rest out:
            f_cpu2 = f_cpu1.replace('with', '')
            f_cpu2 = f_cpu2.replace('2GB Memor', '')
            f_cpu2 = f_cpu2.replace('4GB Memor', '')
            f_cpu2 = f_cpu2.replace('8GB Memor', '')  
            f_cpu2 = f_cpu2.replace('16GB Memor', '')
            f_cpu2 = f_cpu2[1:]

            if len(f_cpu2) > 2:
                fcpu1.append(f_cpu2)
            else:
                fcpu1.append('')
        
        # if 'Processor' is not part of its name:
        elif (f_cpu1.find('Intel') >= 0) or (f_cpu1.find('AMD') >= 0):
            
            # clean out junk:
            f_cpu2 = f_cpu1.replace('Full HD IPS Laptop 8GB 1TB HDD Windows 10 Home ', '')
            f_cpu2 = f_cpu2.replace(' Laptop with 4GB Memory', '')
            f_cpu2 = f_cpu2.replace('"', '')
            f_cpu2 = f_cpu2.replace(' 4GB Memory', '')
            f_cpu2 = f_cpu2.replace(' Halo Keyboard', '')
            f_cpu2 = f_cpu2.replace(' 8GB 512GB SSD Touchscreen Laptop - Grey', '')
            f_cpu2 = f_cpu2.replace('   - Aluminum', '')
            f_cpu2 = f_cpu2.replace('With', 'with')
            
            
            if f_cpu2.find('with') > 0:
                # then retrieve everything after as cpu:
                f_cpu2 = f_cpu2[f_cpu2.find('with')+5:]
                fcpu1.append(f_cpu2)
                
            elif f_cpu2.find('3rd Party Intel'):
                fcpu1.append(f_cpu2)
                
            elif f_cpu2.find('Intel') >= 0:
                f_cpu2 = f_cpu2[f_cpu2.find('Intel'):]
                fcpu1.append(f_cpu2)

            elif f_cpu2.find('AMD') >= 0:
                f_cpu2 = f_cpu2[f_cpu2.find('AMD'):]
                fcpu1.append(f_cpu2)
            
            else:
                fcpu1.append('')
        else:
            fcpu1.append('')

    else:  # x != '':
        fcpu1.append(x)

        
    frnum += 1
    


fcpu1


['Intel Ci5-3210M',
 'Intel Celeorn N2920',
 'Intel Ci5-520M',
 'Intel Ci5-4300u',
 'Ci5-2520M',
 'Intel Celeron N2840',
 'Intel Celeron 1007U',
 'Intel Ci5-3210M',
 'Intel Ci5-520M',
 'Intel Celeron N2840',
 'Intel Ci5-4200M',
 'Intel Ci5-3427U',
 'AMD Ryzen 5 2500U',
 'Intel Ci5-4200U',
 'Intel Ci5-4300M',
 'Intel Ci5-4300U',
 'Intel Ci5-3320M',
 'AMD Ryzen5 3500U',
 'Intel Ci7-8550U',
 'AMD Ryzen R3 2200U',
 'Intel Ci5-8250U',
 'AMD Ryzen 5 2500U',
 'Intel Ci5-8250U',
 'Intel Celeron N2930',
 'AMD A6-9225',
 'AMD Ryzen3 2200U',
 'AMD Ryzen5 2500U',
 'AMD Ryzen 3 3200U',
 'Intel Ci5-8250U',
 'Intel Ci5-8250U',
 'Intel Ci5-8250U',
 'Intel Ci5-2520M',
 'Intel Ci5-8265U',
 'New Intel 9th Generation Intel® Core',
 'Intel Core i5-3230M',
 'Intel Ci5-3320m',
 'Intel Ci7-8565U',
 'AMD A9-9425',
 'Intel Ci7-8565U',
 'Intel Ci7-8565U',
 'Intel Ci5-8250U',
 'Intel Ci7-8565U',
 'Intel i3-8145U',
 'Intel Ci7-8565U',
 'Intel Ci7-8565U',
 'Intel New 9th Generation Intel® Core',
 '9th Generation In

In [18]:
# cpu round 3: look in col 3:
fraw_cpu2 = []
fcpu2 = []
frownum3 = 0


for x in fcpu1:
    if x == '':
        # check for nulls:
        if frys_specs.loc[frownum3, 3] is None:
            fcpu2.append('')

        else:    
            # pull the data from col 3:
            f_cpu3 = frys_specs.loc[frownum3, 3]

            # clean the junk out:
            f_cpu4 = f_cpu3.replace('1GB Memory', '')
            f_cpu4 = f_cpu3.replace('2GB Memor', '')
            f_cpu4 = f_cpu4.replace('4GB Memor', '')
            f_cpu4 = f_cpu4.replace('8GB Memor', '')  
            f_cpu4 = f_cpu4.replace('16GB Memor', '')
            f_cpu4 = f_cpu4.replace('320GB', '')
            f_cpu4 = f_cpu4.replace('256GB', '')
            f_cpu4 = f_cpu4.replace('16GB', '')
            f_cpu4 = f_cpu4.replace('4GB', '')
            f_cpu4 = f_cpu4.replace('1TB', '')
            f_cpu4 = f_cpu4.replace('Hard Drive', '')
            f_cpu4 = f_cpu4.replace('Windows 10', '')
            f_cpu4 = f_cpu4.replace('with', 'With')

            f_cpu4 = f_cpu4[f_cpu4.find('With')+4:]
            f_cpu4 = f_cpu4.replace(' Processor', '')
            f_cpu4 = f_cpu4.replace('y,', '')
            f_cpu4 = f_cpu4.replace('Storage', '')
            f_cpu4 = f_cpu4.replace('GB', '')
            f_cpu4 = f_cpu4.replace('"', '')

            if (f_cpu4.find('Intel') > 0) or (f_cpu4.find('AMD') > 0):
                fcpu2.append(f_cpu4[1:])
            else:
                fcpu2.append('')
                
    else:
        fcpu2.append(x)
    
    frownum3 += 1


fcpu2
    
    
 
    
    

['Intel Ci5-3210M',
 'Intel Celeorn N2920',
 'Intel Ci5-520M',
 'Intel Ci5-4300u',
 'Ci5-2520M',
 'Intel Celeron N2840',
 'Intel Celeron 1007U',
 'Intel Ci5-3210M',
 'Intel Ci5-520M',
 'Intel Celeron N2840',
 'Intel Ci5-4200M',
 'Intel Ci5-3427U',
 'AMD Ryzen 5 2500U',
 'Intel Ci5-4200U',
 'Intel Ci5-4300M',
 'Intel Ci5-4300U',
 'Intel Ci5-3320M',
 'AMD Ryzen5 3500U',
 'Intel Ci7-8550U',
 'AMD Ryzen R3 2200U',
 'Intel Ci5-8250U',
 'AMD Ryzen 5 2500U',
 'Intel Ci5-8250U',
 'Intel Celeron N2930',
 'AMD A6-9225',
 'AMD Ryzen3 2200U',
 'AMD Ryzen5 2500U',
 'AMD Ryzen 3 3200U',
 'Intel Ci5-8250U',
 'Intel Ci5-8250U',
 'Intel Ci5-8250U',
 'Intel Ci5-2520M',
 'Intel Ci5-8265U',
 'New Intel 9th Generation Intel® Core',
 'Intel Core i5-3230M',
 'Intel Ci5-3320m',
 'Intel Ci7-8565U',
 'AMD A9-9425',
 'Intel Ci7-8565U',
 'Intel Ci7-8565U',
 'Intel Ci5-8250U',
 'Intel Ci7-8565U',
 'Intel i3-8145U',
 'Intel Ci7-8565U',
 'Intel Ci7-8565U',
 'Intel New 9th Generation Intel® Core',
 '9th Generation In

In [35]:
# cpu round 4: translations:
fcpu3 = []

for x in fcpu2:
    if (x.find('Pentium') >0):
        fcpu3.append('Intel Pentium')
    
    elif (x.find('Ci5') >0) or (x.find('Ci5-2520M') >0):
        fcpu3.append('Intel Core i5')
    
    elif (x.find('Celeron') >0) or (x.find('Celeorn') >0):
        fcpu3.append('Intel Celeron')
        
    elif (x.find('Ci3') >0):
        fcpu3.append('Intel Core i3')
        
    elif (x.find('Ci7') >0):
        fcpu3.append('Intel Core i7')    

    elif (x.find('Ryzen5') >0) or (x.find('Ryzen 5') >0):
        fcpu3.append('AMD Ryzen 5')

    elif (x.find('Ryzen3') >0) or (x.find('Ryzen 3') >0) or (x.find('Ryzen R3') >0):
        fcpu3.append('AMD Ryzen 3')
        
    elif (x.find('AMD A4') >0):
        fcpu3.append('AMD A4')

    elif (x.find('AMD A9') >0):
        fcpu3.append('AMD A9')

    elif (x.find('AMD A10') >0):
        fcpu3.append('AMD A10')
        
    elif (x.find('AMD R3') >0):
        fcpu3.append('AMD R3')
        
    else:
        fcpu3.append(x)
        
fcpu3   


fcpu4 = []

for x in fcpu3:
    if (x.find('-') > 0):
        fcpu4.append(x[0:x.find('-')])
    else:
        fcpu4.append(x)
        
fcpu4


frys_specs['cpu'] = fcpu4

frys_specs  

,0,1,2,3,4,5,6,7,8,screensize,cpu
0,696897751172,"PB 6470B 14""",Laptop With Intel Ci5-3210M Processor,8GB Memory,500GB Hard Drive,DVD-ROM,Windows 10,None,None,14,Intel Core i5
1,696897764240,"TP 11e 11.6""",Chromebook With Intel Celeorn N2920 Processor,4GB Memory,16GB Storage,Chrome OS,None,None,None,11.6,Intel Celeron
2,696897763816,"EB 8440P 14""",Laptop With Intel Ci5-520M Processor,4GB Memory,160GB Hard Drive,DVD Rom,Windows 10,None,None,14,Intel Core i5
3,696897764264,"TP X240 12.5""",Laptop With Intel Ci5-4300u Processor,8GB Memory,500GB Hard Drive,Windows 10,None,None,None,12.5,Intel Core i5
4,696897743108,"T420 14.1""",Laptop With Ci5-2520M Processor,4GB Memory,250GB Hard Drive and Windows 10,None,None,None,None,14.1,Ci5
5,696897764257,"N21 11.6""",Chromebook With Intel Celeron N2840 Processor,4GB Memory,16GB Storage,Chrome OS,None,None,None,11.6,Intel Celeron
6,696897751226,"X131E 11.6""",Laptop With Intel Celeron 1007U Processor,4GB Memory,320GB Hard Drive,Windows 10 Home,None,None,None,11.6,Intel Celeron
7,696897750199,"Thinkpad T430 14""",Laptop With Intel Ci5-3210M Processor,8GB Memory,240GB SSD,Windows 10 Pro,None,None,None,14,Intel Core i5
8,696897743115,"T410 14.1""",Laptop With Intel Ci5-520M Processor,4GB Memory,160GB Hard Drive and Windows 10,None,None,None,None,14.1,Intel Core i5
9,724962748779,"G4 11.6""",Chromebook With Intel Celeron N2840 Processor,2GB Memory,16GB SSD,Chrome OS,None,None,None,11.6,Intel Celeron


## RAM

In [36]:
ram_string_arr  = frys_df['modified_title']

ram_num_arr = []

for x in ram_string_arr:
    raw_mem = x.replace('memory', 'Memory')
    raw_mem = raw_mem.replace(',', ' ')
    
    if raw_mem.find('Memory') >=0:
        #clean the data:
        ramNo = raw_mem[raw_mem.find('Memory')-5:raw_mem.find('Memory')-3]
        ramNo = ramNo.replace(' ','')
        ram_num_arr.append(int(ramNo))
    
    elif raw_mem.find('8 GB DDR4 SDRAM') >= 0:
        ramNo = 8
        ram_num_arr.append(int(ramNo))
    
    elif raw_mem.find('RAM') >= 0:
        #clean the data:
        ramNo = raw_mem[raw_mem.find('RAM')-4:raw_mem.find('RAM')-3]
        ram_num_arr.append(int(ramNo))
    
    elif raw_mem.find('Full HD IPS Laptop 8GB') >= 0:
        ramNo = 8
        ram_num_arr.append(int(ramNo))
    
    else:
        ram_num_arr.append('')

ram_num_arr

# ram_string_arr[237]
  
frys_specs['ram'] = ram_num_arr

frys_specs  


,0,1,2,3,4,5,6,7,8,screensize,cpu,ram
0,696897751172,"PB 6470B 14""",Laptop With Intel Ci5-3210M Processor,8GB Memory,500GB Hard Drive,DVD-ROM,Windows 10,None,None,14,Intel Core i5,8
1,696897764240,"TP 11e 11.6""",Chromebook With Intel Celeorn N2920 Processor,4GB Memory,16GB Storage,Chrome OS,None,None,None,11.6,Intel Celeron,4
2,696897763816,"EB 8440P 14""",Laptop With Intel Ci5-520M Processor,4GB Memory,160GB Hard Drive,DVD Rom,Windows 10,None,None,14,Intel Core i5,4
3,696897764264,"TP X240 12.5""",Laptop With Intel Ci5-4300u Processor,8GB Memory,500GB Hard Drive,Windows 10,None,None,None,12.5,Intel Core i5,8
4,696897743108,"T420 14.1""",Laptop With Ci5-2520M Processor,4GB Memory,250GB Hard Drive and Windows 10,None,None,None,None,14.1,Ci5,4
5,696897764257,"N21 11.6""",Chromebook With Intel Celeron N2840 Processor,4GB Memory,16GB Storage,Chrome OS,None,None,None,11.6,Intel Celeron,4
6,696897751226,"X131E 11.6""",Laptop With Intel Celeron 1007U Processor,4GB Memory,320GB Hard Drive,Windows 10 Home,None,None,None,11.6,Intel Celeron,4
7,696897750199,"Thinkpad T430 14""",Laptop With Intel Ci5-3210M Processor,8GB Memory,240GB SSD,Windows 10 Pro,None,None,None,14,Intel Core i5,8
8,696897743115,"T410 14.1""",Laptop With Intel Ci5-520M Processor,4GB Memory,160GB Hard Drive and Windows 10,None,None,None,None,14.1,Intel Core i5,4
9,724962748779,"G4 11.6""",Chromebook With Intel Celeron N2840 Processor,2GB Memory,16GB SSD,Chrome OS,None,None,None,11.6,Intel Celeron,2


## HD

In [37]:
raw_frys_hd_arr = frys_df['modified_title']

frys_hd_arr = []

for x in raw_frys_hd_arr:
    if x.find('Hard Drive') >= 0:
        # clean data:
        hd = x[x.find('Hard Drive')-6: x.find('Hard Drive')-1]
        hd = hd.replace(', ', ' ')
        hd = hd.replace(',', ' ')
        hd = hd.strip()
        
        if hd.find('TB') >= 0:
            hd = hd[hd.find('TB')-1:hd.find('TB')]
            hd = int(hd) * 1000  # to convert it to GB
            frys_hd_arr.append(int(hd))
        elif hd.find('GB') >= 0:
            hd = hd[0:hd.find('GB')]
            frys_hd_arr.append(int(hd))
    
    
    elif x.find('HD') >= 0:
         # clean data:
        hd = x[x.find('HD')-6:x.find('HD')+2]

        #clean data
        if hd.find('TB') >= 0:
            hd = hd[hd.find('TB')-1:hd.find('TB')]
            hd = int(hd) * 1000 # to convert to GB
            frys_hd_arr.append(int(hd))
        elif hd.find('GB') >=0:
            hd = hd[0:hd.find('GB')]
            frys_hd_arr.append(int(hd))

        else:
            frys_hd_arr.append('')
            
            

    elif x.find('SSD') >= 0:
         # clean data:
        hd = x[x.find('SSD')-6:x.find('SSD')+3]
        
        if hd.find('GB') >=0:
            hd = hd[0:hd.find('GB')]
            frys_hd_arr.append(int(hd))
        else:
            frys_hd_arr.append('')
                        
            

    elif x.find('Storage') >= 0:
         # clean data:
        hd = x[x.find('Storage')-6:x.find('Storage')-1]

        #clean data
        if hd.find('GB') >=0:
            hd = hd[0:hd.find('GB')]
            frys_hd_arr.append(int(hd))
        else:
            frys_hd_arr.append('')
                        
    
    else:
        frys_hd_arr.append('')

frys_hd_arr

  
frys_specs['hd'] = frys_hd_arr

frys_specs  


,0,1,2,3,4,5,6,7,8,screensize,cpu,ram,hd
0,696897751172,"PB 6470B 14""",Laptop With Intel Ci5-3210M Processor,8GB Memory,500GB Hard Drive,DVD-ROM,Windows 10,None,None,14,Intel Core i5,8,500
1,696897764240,"TP 11e 11.6""",Chromebook With Intel Celeorn N2920 Processor,4GB Memory,16GB Storage,Chrome OS,None,None,None,11.6,Intel Celeron,4,16
2,696897763816,"EB 8440P 14""",Laptop With Intel Ci5-520M Processor,4GB Memory,160GB Hard Drive,DVD Rom,Windows 10,None,None,14,Intel Core i5,4,160
3,696897764264,"TP X240 12.5""",Laptop With Intel Ci5-4300u Processor,8GB Memory,500GB Hard Drive,Windows 10,None,None,None,12.5,Intel Core i5,8,500
4,696897743108,"T420 14.1""",Laptop With Ci5-2520M Processor,4GB Memory,250GB Hard Drive and Windows 10,None,None,None,None,14.1,Ci5,4,250
5,696897764257,"N21 11.6""",Chromebook With Intel Celeron N2840 Processor,4GB Memory,16GB Storage,Chrome OS,None,None,None,11.6,Intel Celeron,4,16
6,696897751226,"X131E 11.6""",Laptop With Intel Celeron 1007U Processor,4GB Memory,320GB Hard Drive,Windows 10 Home,None,None,None,11.6,Intel Celeron,4,320
7,696897750199,"Thinkpad T430 14""",Laptop With Intel Ci5-3210M Processor,8GB Memory,240GB SSD,Windows 10 Pro,None,None,None,14,Intel Core i5,8,240
8,696897743115,"T410 14.1""",Laptop With Intel Ci5-520M Processor,4GB Memory,160GB Hard Drive and Windows 10,None,None,None,None,14.1,Intel Core i5,4,160
9,724962748779,"G4 11.6""",Chromebook With Intel Celeron N2840 Processor,2GB Memory,16GB SSD,Chrome OS,None,None,None,11.6,Intel Celeron,2,16


## Final Frys Table

In [38]:
frys_specs.head()

final_frys_specs = pd.DataFrame(frys_specs[[0, 'screensize', 'cpu', 'ram', 'hd']])
final_frys_specs.rename(columns={0: 'upc'}, inplace = True)
final_frys_specs.head()

,upc,screensize,cpu,ram,hd
0,696897751172,14,Intel Core i5,8,500
1,696897764240,11.6,Intel Celeron,4,16
2,696897763816,14,Intel Core i5,4,160
3,696897764264,12.5,Intel Core i5,8,500
4,696897743108,14.1,Ci5,4,250


In [39]:
frys_df.head()
shorter_frys_df = frys_df[['brand', 'link', 'model', 'price', 'title', 'upc']]
shorter_frys_df.head()

,brand,link,model,price,title,upc
0,HP,https://www.frys.com/product/9800005;jsessioni...,PB6470BCi5-3210M,399.0,"HP PB 6470B 14"" Refurbished Laptop With Intel ...",696897751172
1,Lenovo,https://www.frys.com/product/9914936;jsessioni...,TP11eCel.N2920,199.0,"Lenovo TP 11e 11.6"" Refurbished Chromebook Wit...",696897764240
2,HP,https://www.frys.com/product/9911496;jsessioni...,EB8440Pi5-520M,299.0,"HP EB 8440P 14"" Refurbished Laptop With Intel ...",696897763816
3,Lenovo,https://www.frys.com/product/9914966;jsessioni...,TPX240i5-4300U,399.0,"Lenovo TP X240 12.5"" Refurbished Laptop With I...",696897764264
4,Lenovo,https://www.frys.com/product/9721554;jsessioni...,T420Ci5-2520M,349.0,"Lenovo T420 14.1"" Refurbished Laptop With Ci5-...",696897743108


In [40]:
frys_merged1_df = shorter_frys_df.merge(final_frys_specs, on='upc', how='left')
final_frys_df = frys_merged1_df[['brand', 'model', 'screensize', 'cpu', 'ram', 'hd', 'upc', 'price', 'link', 'title']]


# Add the load date:
now = DT.datetime.now()

final_frys_df['lastUpdated'] = now.strftime("%Y-%m-%d %H:%M")

final_frys_df.head()

C:\Users\Rika\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,brand,model,screensize,cpu,ram,hd,upc,price,link,title,lastUpdated
0,HP,PB6470BCi5-3210M,14,Intel Core i5,8,500,696897751172,399.0,https://www.frys.com/product/9800005;jsessioni...,"HP PB 6470B 14"" Refurbished Laptop With Intel ...",2019-10-02 12:40
1,Lenovo,TP11eCel.N2920,11.6,Intel Celeron,4,16,696897764240,199.0,https://www.frys.com/product/9914936;jsessioni...,"Lenovo TP 11e 11.6"" Refurbished Chromebook Wit...",2019-10-02 12:40
2,HP,EB8440Pi5-520M,14,Intel Core i5,4,160,696897763816,299.0,https://www.frys.com/product/9911496;jsessioni...,"HP EB 8440P 14"" Refurbished Laptop With Intel ...",2019-10-02 12:40
3,Lenovo,TPX240i5-4300U,12.5,Intel Core i5,8,500,696897764264,399.0,https://www.frys.com/product/9914966;jsessioni...,"Lenovo TP X240 12.5"" Refurbished Laptop With I...",2019-10-02 12:40
4,Lenovo,T420Ci5-2520M,14.1,Ci5,4,250,696897743108,349.0,https://www.frys.com/product/9721554;jsessioni...,"Lenovo T420 14.1"" Refurbished Laptop With Ci5-...",2019-10-02 12:40


# Load: Loading both Frys and BestBuy data into PostgreSQL

In [41]:
###############################################################
# LOAD: 
###############################################################


# change to your username:password@localhost:5432/laptop_db:
connection = "postgres:helloworld@localhost:5432/laptop_db"
engine = create_engine(f'postgresql://{connection}')

# add the FRYS dataframe to the table:
final_frys_df.to_sql(name="frys_laptops", con=engine, if_exists='replace', index=False)



# add the BEST BUY dataframe to the table:
final_bestbuy_df.to_sql(name="bestbuy_laptops", con=engine, if_exists='replace', index=False)


# combine the 2 tables:
final_frys_df['store'] = "Fry's"
final_bestbuy_df['store'] = "Bestbuy"

combo_df_arr = [final_bestbuy_df, final_frys_df]
final_laptops_df = pd.concat(combo_df_arr)
final_laptops_df['upc_store'] = final_laptops_df['upc'].str.cat(final_laptops_df['store'], sep = '_')
final_laptops_df = final_laptops_df.dropna(subset=['upc'])

final_laptops_df.head()


final_laptops_df.to_sql(name="both_laptops", con=engine, if_exists='replace', index=False)


with engine.connect() as con:
    try:
        con.execute('ALTER TABLE bestbuy_laptops ADD CONSTRAINT primary_key1 PRIMARY KEY(upc);')
    except:
        print('bb already done')
    try:
        con.execute('ALTER TABLE frys_laptops ADD CONSTRAINT primary_key2 PRIMARY KEY(upc);')
    except:
        print('frys already done')
    try:
        con.execute('ALTER TABLE both_laptops ADD CONSTRAINT primary_key3 PRIMARY KEY(upc_store);')
    except:
        print('both already done')
        
# VALIDATE TABLE BUILD:
engine.table_names()

C:\Users\Rika\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['bestbuy_laptops',
 'frys_laptops',
 'both_laptops',
 'bestbuy_locations',
 'frys_locations',
 'both_locations']